Import Packages

In [134]:
import requests
import re
import pandas as pd
import urllib.parse
import codecs
import time
from random import randint
from pyquery import PyQuery as pq

Log into Danbooru

In [2]:
session_requests = requests.session()
auth_token = None
    
login_url = 'https://danbooru.donmai.us/session/new'
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
result = session_requests.get(login_url, headers = headers)

login_page = pq(result.text)
authenticity_token = urllib.parse.quote(login_page('input').attr('value'),safe='')
#print(authenticity_token)

login_data = {
    'authenticity_token': authenticity_token,
    'url': '',
    'name': 'komeiji_nori',
    'password': ,
    'remember': '1',   
    'commit': 'Submit'
}

resp = session_requests.post(url = 'https://danbooru.donmai.us/session', data = login_data, headers = headers)

Get Wiki Page

In [ ]:
# wiki_url = 'https://danbooru.donmai.us/wiki_pages/62620'
# wiki_page_data = session_requests.get(wiki_url)

Define function to parse character list

In [82]:
# def chara_map_creation(wiki_page_data):
#     chara_map = {}
#     wiki_page_query = pq(wiki_page_data.text)
#     for tag_li in wiki_page_query('li'):
#         li_query = pq(tag_li)
#         li_a_class = li_query('a').attr('class') or ''
#         if 'dtext-link' in li_a_class:
#             li_text = li_query.text()
#             try:
#                 chara_name = re.search('(?!.* /)(?<=/ ).*',li_text).group(0)
#             except:
#                 try:
#                     chara_name = li_query('a').text()
#                 except:
#                     chara_name = ''
#             chara_url = li_query('a').attr('href') or ''
#             if chara_url:
#                 chara_tag = re.search('(?<=title=).*',chara_url).group(0)
#             if chara_name:
#                 chara_map[chara_name] = chara_tag
#     return chara_map

Output Chara Dictionary. Cleaned and edited outside of python

In [94]:
# kantai_chara = chara_map_creation(wiki_page_data)
# dict_file = codecs.open('kantai_chara_dict.txt', 'w', 'utf-8')

# for i in kantai_chara:
#     dict_file.write('{}\t{}\n'.format(i,kantai_chara[i]))

# dict_file.close()

In [95]:
# Test Block
# new_url = 'https://danbooru.donmai.us/counts/posts?tags=shimakaze_(kantai_collection)+rating:safe+parent:None+amatsukaze_(kantai_collection)'
# page_data = session_requests.get(new_url)
# page_query = pq(page_data.text)
# for tag_div in page_query('div'):
#     div_query = pq(tag_div)
#     div_id = div_query.attr('id')
#     if div_id == 'a-posts':
#         count = div_query.text()
#         count = re.findall(r'\d+', count)[0]
#         print(count)

1001


In [152]:
dict_file = codecs.open('kantai_chara_dict_final.txt', 'r', 'utf-8')
chara_dict = {}

for line in dict_file:
    name_pair = (line[:-1]).split('\t')
    chara_dict[name_pair[0]] = name_pair[1]

print(chara_dict['叢雲'])
print(chara_dict['島風'])

murakumo_%28kantai_collection%29
shimakaze_%28kantai_collection%29


In [139]:
def count_danbooru(key):
    
    output_dict = {}
    parent_flag = ['all','parent%3Anone']
    rating_flag = ['all','rating%3Aexplicit','rating%3Aquestionable','rating%3Asafe']
    
    for i in parent_flag:
        for j in rating_flag:
            
            #build query url
            chara_url = 'https://danbooru.donmai.us/counts/posts?tags=+-official_art'
            if i != 'all':
                chara_url = chara_url + '+' + i
            if j != 'all':
                chara_url = chara_url + '+' + j
            if key != '島風':
                chara_url = chara_url + '+' + chara_dict[key]
            else:
                chara_url = chara_url + '+' + chara_dict[key] + '+-shimakaze_%28kantai_collection%29_%28cosplay%29'

            #get query url and count
            chara_count_page = session_requests.get(chara_url)
            chara_count_query = pq(chara_count_page.text)

            for tag_div in chara_count_query('div'):
                div_query = pq(tag_div)
                div_id = div_query.attr('id')
                if div_id == 'a-posts':
                    count = div_query.remove('a').text()
                    count = re.findall(r'\d+', count)[0]

            output_dict_key = (key, urllib.parse.unquote(i), urllib.parse.unquote(j))
            output_dict[output_dict_key] = count
    
            # Waits before sending next request
            time.sleep(10 + randint(0,10))
            
    return output_dict

In [153]:
final_result = {}

for i in chara_dict:
    a = count_danbooru(i)
    final_result = {**final_result, **a}

final_output_file = codecs.open('final.txt', 'w', 'utf-8')

for i in final_result:
    chara_name, parent_flag, ratings_flag = i
    final_output_file.write('{}\t{}\t{}\t{}\n'.format(chara_name, parent_flag, ratings_flag, final_result[i]))

final_output_file.close()